<a href="https://colab.research.google.com/github/Vishvam10/LLM_RAG/blob/master/RAG_Mistral_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Imports and Installs**

In [1]:
!pip install -q  -U torch datasets transformers tensorflow langchain sentence_transformers faiss-cpu
!pip install -q  accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [27]:
import os
import torch

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)

from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader, TextLoader
from langchain.schema.document import Document

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

import warnings
warnings.filterwarnings("ignore")

## **Model Initialization**

In [3]:
#################################################################
# Tokenizer and bitsandbytes parameters
#################################################################

model_name='mistralai/Mistral-7B-Instruct-v0.1'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [4]:
#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [5]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 262410240
all model parameters: 3752071168
percentage of trainable model parameters: 6.99%


In [6]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    repetition_penalty=1.0,
    return_full_text=True,
    max_new_tokens=2048,
)

In [7]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

## **RAG Setup**

In [ ]:
# !playwright install
# !playwright install-deps

In [ ]:
# import nest_asyncio
# nest_asyncio.apply()

# # Articles to index
# articles = [
#   'https://www.w3schools.com/css/css_grid.asp',
#   'https://www.w3schools.com/css/css_grid_container.asp',
#   'https://www.w3schools.com/css/css_grid_item.asp',
#   'https://tailwindcss.com/docs/installation',
# ]

# loader = AsyncChromiumLoader(articles)
# docs = loader.load()

# # Converts HTML to plain text
# html2text = Html2TextTransformer()
# docs_transformed = html2text.transform_documents(docs)

# # Chunk text
# text_splitter = CharacterTextSplitter(chunk_size=1024,
#                                       chunk_overlap=0)
# chunked_documents = text_splitter.split_documents(docs_transformed)

# # Load chunked documents into the FAISS index
# db = FAISS.from_documents(chunked_documents,
#                           HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

# retriever = db.as_retriever()

In [9]:
embeddings = HuggingFaceEmbeddings()
db = FAISS.load_local(folder_path='./bootstrap_rag', embeddings=embeddings)
retriever = db.as_retriever()

In [10]:
print(retriever)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e0778263c70>


In [ ]:
# db.save_local("bootstrap_embeddings")

## **Prompt Templating**

In [11]:
prompt_template = """
### [INST] Instruction: You are a UI generating assistant. You'll carefully analyse the input prompt and then create the UI using React and CSS3. Use CSS classes wherever you can and define them in a styles tag.
Infer things like background colors, shadows, borders, etc from the nature of the UI component. Follow the input prompt thoroughly. Print only the code that is asked in the input prompt
and nothing else (no explanation or comments or things like "here you go", "here's the code that you asked for", etc)

{context}

### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

### **Test Output**

##### **Without RAG**

In [ ]:
question = '''
Generate a simple card with a heading and some sample text (use lorem ipsum).
Give the react component as output and follow the styles given as context.
'''

context = ''

result = llm_chain.invoke({"context": context, "question": question})

print(result['text'].replace('\n', ''))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


```jsximport React from'react';import './styles.css';const Card = ({ heading, text }) => {  return (    <div className="card">      <h2 className="heading">{heading}</h2>      <p className="text">{text}</p>    </div>  );};export default Card;```


##### **With RAG**

In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

question = '''
Generate a simple card with a heading and some sample text. Infer from the context the heading and a sample text.
Give the react component as output and follow the styles given as context.
'''

result = rag_chain.invoke(question)

print(result['text'])

## **UI Generation**

### **Simple Components**

##### **Card Component**

In [ ]:
question = '''
Generate a simple image card. The top of the card should have an image with source as : https://a-z-animals.com/media/2022/11/shutterstock_606517310-1024x650.jpg.
The bottom of the card should contain the text "Sample Image". Give the react component as output and follow the styles given in the context.
'''

result = rag_chain.invoke(question)

print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```javascript
import React from'react';
import { addComponents } from 'tailwindcss-in-js';

const Card = ({ img, imgAlt }) => {
  const classes = addComponents({
    '.card': {
      backgroundColor: '#fff',
      borderRadius: '.25rem',
      boxShadow: '0 2px 4px rgba(0,0,0,0.2)',
      '&:hover': {
        boxShadow: '0 10px 15px rgba(0,0,0,0.2)',
      },
      '@media (min-width: 500px)': {
        borderRadius: '.5rem',
      },
    },
    '.card-image': {
      width: '100%',
      height: 'auto',
      objectFit: 'cover',
    },
    '.card-text': {
      textAlign: 'center',
      fontSize: '1.2rem',
      fontWeight: 'bold',
      color: '#000',
      marginTop: '1rem',
    },
  });

  return (
    <div className={classes.card}>
      <img className={classes.cardImage} src={img} alt={imgAlt} />
      <div className={classes.cardText}>Sample Image</div>
    </div>
  );
};

export default Card;
```


In [12]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

question = '''
Generate a simple image card. The top of the card should have an image with source as : https://shorturl.at/kCOZ2.
The bottom of the card should contain the text "Sample Image".
Give the react component as output and follow the styles given in the context.
DO NOT USE ANY PROPS instead hardcode the values wherever necessary.
'''

result = rag_chain.invoke(question)

print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```
import React from'react';
import './ImageCard.css';

const ImageCard = () => {
  return (
    <div className="image-card">
      <img src="https://shorturl.at/kCOZ2" alt="Sample Image" className="image" />
      <p className="text">Sample Image</p>
    </div>
  );
};

export default ImageCard;
```

```
.image-card {
  display: flex;
  flex-direction: column;
  align-items: center;
  justify-content: center;
  width: 300px;
  height: 400px;
  background-color: #f2f2f2;
  border-radius: 10px;
  box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.2);
}

.image {
  width: 100%;
  height: 200px;
  object-fit: cover;
  border-radius: 10px;
}

.text {
  font-size: 18px;
  text-align: center;
  color: #333;
  margin-top: 10px;
}
```


In [14]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

question = '''
Generate a simple card. The top of the card should have an image with source as : https://shorturl.at/kCOZ2.
The bottom of the card should contain the text "Sample Image". This is the heading (at most 20px).
Below that, there should be a smaller text (1.5 times smaller) that says "In publishing and graphic design, Lorem ipsum
is a placeholder text commonly used to demonstrate the visual form of a document or a typeface without relying on meaningful content." Below that, there should be
a simple button that says 'Read More'.

Give the react component as output and follow the styles given in the context. DO NOT USE ANY PROPS
instead hardcode the values wherever necessary.
'''

result = rag_chain.invoke(question)

print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```
import React from'react';
import './Card.css';

const Card = () => {
  return (
    <div className="card">
      <img src="https://shorturl.at/kCOZ2" alt="Sample Image" className="card-image" />
      <div className="card-text">
        <h2 className="card-heading">Sample Image</h2>
        <p className="card-body">
          In publishing and graphic design, Lorem ipsum is a placeholder text commonly used to demonstrate the visual form of a document or a typeface without relying on meaningful content.
        </p>
        <button className="card-button">Read More</button>
      </div>
    </div>
  );
};

export default Card;
```

```
.card {
  display: flex;
  flex-direction: column;
  align-items: center;
  background-color: #f2f2f2;
  border-radius: 5px;
  box-shadow: 0 0 5px rgba(0, 0, 0, 0.2);
  margin: 10px;
  width: 300px;
  height: 400px;
}

.card-image {
  width: 100%;
  height: 200px;
  object-fit: cover;
}

.card-text {
  display: flex;
  flex-direction: column;
  align

In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

question = '''
Generate a simple product card. The top of the card should have an image with source as : https://shorturl.at/kmzN2. For the bottom part, use a 2-column layout.
The bottom of card should contain the following : "Sample Shoes" as heading, "Sample Company" as subheading in a lighter text and "$120.00" as price
in a bold font. All of them should be below each other and should be in the first column. In the second column, right beside the heading, include a button that says "+".

Give the react component as output and follow the styles given in the context. DO NOT USE ANY PROPS instead hardcode the values wherever necessary.
'''

result = rag_chain.invoke(question)

print(result['text'])

##### **Input Component**

In [15]:
question = '''
Generate a simple form component. Give the form a title (heading) of "Contact Form". This should follow a 12 column layout. Inside this form component have the following in separate rows :
- input component with placeholder as "Sample" and label as "Name". It should spans 12 columns
- input component with placeholder as "sample@gmail.com" and label as "Email". It should spans 12 columns
- one button component with text "Submit" at the bottom of the container.

Make the form responsive and make sure maximum dimensions are height = 800px and width = 500px.

Give the react component (JS not typescript) as output and follow the styles given in the context.
DO NOT USE ANY PROPS instead hardcode the values wherever necessary.
Define all the functions that you use in the component definition.
DO NOT USE ANY OTHER ADDITIONAL LIBRARIES for icons, etc.

'''

result = rag_chain.invoke(question)

print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```jsx
import React from'react';
import './ContactForm.css';

const ContactForm = () => {
  return (
    <div className="contact-form-container">
      <div className="contact-form-title">Contact Form</div>
      <div className="contact-form-row">
        <div className="contact-form-column">
          <input
            type="text"
            placeholder="Sample"
            className="contact-form-input"
          />
          <label className="contact-form-label">Name</label>
        </div>
      </div>
      <div className="contact-form-row">
        <div className="contact-form-column">
          <input
            type="email"
            placeholder="sample@gmail.com"
            className="contact-form-input"
          />
          <label className="contact-form-label">Email</label>
        </div>
      </div>
      <div className="contact-form-row">
        <div className="contact-form-column">
          <button className="contact-form-button">Submit</button>
        </div>


In [17]:
question = '''
Generate a simple form component. Give the form a title (heading) of "Contact Form". This should follow a 12 column layout and should be left aligned.
Inside this form component have the following :
first row :
- input component with placeholder as "Sample" and label as "Name". It should spans 6 columns
- input component with placeholder as "sample@gmail.com" and label as "Email". It should spans 6 columns
second row :
- input component with placeholder as "Sample Address" and label as "Address". It should spans 8 columns
- input component with placeholder as "Sample PIN" and label as "Pincode". It should spans 4 columns
last row :
- one button component with text "Submit". It should span 12 columns.

Make the form responsive and make sure maximum dimensions are height = 800px and width = 500px with decent padding.
Give the react component (JS not typescript) as output and follow the styles given in the context.
Define all the functions that you use in the component definition.
DO NOT USE ANY PROPS instead hardcode the values wherever necessary.

ALLOWED : import React from'react';
ALLOWED : import "styles.css";

NO OTHER IMPORTS ARE ALLOWED.
FOLLOW THE COLUMN SPANS USING BOOTSTRAP FLEX-BOX OR GRID.
GIVE PROPER PADDINGS AND MARGIN WHEREVER NECESSARY SO THAT THE UI LOOKS GOOD.

'''

result = rag_chain.invoke(question)

print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```jsx
import React from'react';
import './styles.css';

const ContactForm = () => {
  return (
    <div className="container">
      <div className="row">
        <div className="col-md-12">
          <h1 className="text-center mb-5">Contact Form</h1>
          <form className="form-group">
            <div className="row">
              <div className="col-md-6">
                <label className="form-label">Name</label>
                <input type="text" className="form-control" placeholder="Sample" />
              </div>
              <div className="col-md-6">
                <label className="form-label">Email</label>
                <input type="email" className="form-control" placeholder="sample@gmail.com" />
              </div>
            </div>
            <div className="row">
              <div className="col-md-8">
                <label className="form-label">Address</label>
                <input type="text" className="form-control" placeholder="Sample Address" />
  

### **Complex Components**

##### **Pricing Plan**

In [18]:
question = '''
Generate a pricing page component. This should be a section. The title should say "Pricing". Below that,
in a smaller font-size, there should a copy that says "Lorem ipsum may be used the final copy is available. Lorem ipsum may be used the final copy is available".
Inside that section, we should have 3 pricing cards.
The pricing card contains a header that has the tiers "Free", "Pro", "Ultra". This is an accent text.
This should be separated by a line. Below that, there should be the price "0$ / month", "$8 / month", "$20 / month".
This should have the heading font size. Below that, include the following text as a list for each of the cards :
"Lorem ipsum may be used the final copy is available."
"as a placeholder before"
"sample text here"
Prettify the list and make sure the colors are pleasing to the ease.
Below that there should be a CTA button that says "Download Now", "Buy Now", "Contact Sales" in those 3 pricing cards.
Have a light grey as background and make the card white with decent shadows that look good.

Give the react component (JS not typescript) as output and follow the styles given in the context.
Define all the functions that you use in the component definition.
DO NOT USE ANY PROPS instead hardcode the values wherever necessary.

ALLOWED : import React from'react';
ALLOWED : import "styles.css";

NO OTHER IMPORTS ARE ALLOWED.
FOLLOW THE COLUMN SPANS USING BOOTSTRAP FLEX-BOX OR GRID.
GIVE PROPER PADDINGS AND MARGIN WHEREVER NECESSARY SO THAT THE UI LOOKS GOOD.
GIVE PROPER FONT-SIZE, FONT-COLOR AND OTHER FONT RELATED THINGS SO THAT THE UI LOOKS GOOD
GIVE PROPER COLORS, SHADOWS, ACCENTS, ETC SO THAT THE UI LOOKS GOOD.

'''

result = rag_chain.invoke(question)

print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```jsx
import React from'react';
import './styles.css';

const PricingPage = () => {
  return (
    <section className="pricing-page">
      <h1 className="pricing-page-title">Pricing</h1>
      <p className="pricing-page-copy">Lorem ipsum may be used the final copy is available. Lorem ipsum may be used the final copy is available.</p>
      <div className="pricing-cards">
        <div className="pricing-card">
          <h2 className="pricing-card-header">Free</h2>
          <p className="pricing-card-price">0$ / month</p>
          <ul className="pricing-card-list">
            <li>Lorem ipsum may be used the final copy is available.</li>
            <li>as a placeholder before</li>
            <li>sample text here</li>
          </ul>
          <button className="pricing-card-cta">Download Now</button>
        </div>
        <div className="pricing-card">
          <h2 className="pricing-card-header">Pro</h2>
          <p className="pricing-card-price">$8 / month</p>
          <ul 

##### **List Group**

In [19]:
question = '''
Generate a list group component. Have a scrollable container that has atleast 5 lists in it.
The list (row-wise) should contain an small rounded avatar (img source : https://shorturl.at/jntJT) and
a container (flex-col) that contains a title that says "Lorem ipsum" and a description (with accent color) that says "Sample description".
The title should be aligned with the top of the avatar and the description should align with the the bottom of the avatart

Have a light grey as background and make the list white with decent shadows that look good.
Makes sure the scrollbar looks good.

Give the react component (JS not typescript) as output and follow the styles given in the context.
Define all the functions that you use in the component definition.
DO NOT USE ANY PROPS instead hardcode the values wherever necessary.

ALLOWED : import React from'react';
ALLOWED : import "styles.css";

NO OTHER IMPORTS ARE ALLOWED.
FOLLOW THE COLUMN SPANS USING BOOTSTRAP FLEX-BOX OR GRID.
GIVE PROPER PADDINGS AND MARGIN WHEREVER NECESSARY SO THAT THE UI LOOKS GOOD.
GIVE PROPER FONT-SIZE, FONT-COLOR AND OTHER FONT RELATED THINGS SO THAT THE UI LOOKS GOOD
GIVE PROPER COLORS, SHADOWS, ACCENTS, ETC SO THAT THE UI LOOKS GOOD.

'''

result = rag_chain.invoke(question)

print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```jsx
import React from'react';
import './styles.css';

const ListGroup = () => {
  return (
    <div className="container">
      <div className="row">
        <div className="col-md-4">
          <div className="list-group">
            <div className="list-group-item">
              <img src="https://shorturl.at/jntJT" className="avatar" />
              <div className="list-group-item-content">
                <h3 className="title">Lorem ipsum</h3>
                <p className="description">Sample description</p>
              </div>
            </div>
            <div className="list-group-item">
              <img src="https://shorturl.at/jntJT" className="avatar" />
              <div className="list-group-item-content">
                <h3 className="title">Lorem ipsum</h3>
                <p className="description">Sample description</p>
              </div>
            </div>
            <div className="list-group-item">
              <img src="https://shorturl.at/jntJT" cl

##### **Timeline**

In [21]:
question = '''
Generate a timeline component. It should be in a vertical fashion. The timeline should be a vertical line that stretches for 75% of the height.
Every timestamp should have a big dot on the timeline and should be evenly split across the timeline.
While placing the timestamps, start from the top and place them from top to bottom in a evenly spaced manner.
There should be 3 timestamps : "Jan 1, 2024", "May 1, 2024", "Jul 2, 2024". These dates should be placed horizontal to their correcsponding dots.
These are accent fonts (so have lighter font-color and smaller font-size).
Below that, there should be the title (this is the heading but have a small font anyway) that should say "Title 1", "Title 2" and "Title 3".
Below the title, there should be a small description that says " Lorem ipsum is a placeholder text commonly used to demonstrate the visual form of a document or a typeface without relying on meaningful content." for each of the timestamps.
In the timestamp corresponding to "Jan 1, 2024", have a CTA button below the description that says "Learn More"

Leave some horizontal margin from the timestamp dot to the timestamp content.
Make sure the title, description, button, etc (the content of the timestamp) is presented neatly with proper margins and paddings
Use a light grey background and proper font colors to make the UI look good.


Give the react component (JS not typescript) as output and follow the styles given in the context.
Define all the functions that you use in the component definition.
DO NOT USE ANY PROPS instead hardcode the values wherever necessary.

ALLOWED : import React from'react';
ALLOWED : import "styles.css";

NO OTHER IMPORTS ARE ALLOWED.
FOLLOW THE COLUMN SPANS USING BOOTSTRAP FLEX-BOX OR GRID.
GIVE PROPER PADDINGS AND MARGIN WHEREVER NECESSARY SO THAT THE UI LOOKS GOOD.
GIVE PROPER FONT-SIZE, FONT-COLOR AND OTHER FONT RELATED THINGS SO THAT THE UI LOOKS GOOD
GIVE PROPER COLORS, SHADOWS, ACCENTS, ETC SO THAT THE UI LOOKS GOOD.

'''

result = rag_chain.invoke(question)

print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```jsx
import React from'react';
import './styles.css';

const Timeline = () => {
  return (
    <div className="timeline-container">
      <div className="timeline-line"></div>
      <div className="timeline-dots">
        <div className="timeline-dot" style={{ left: '0%' }}>
          <div className="timeline-content">
            <div className="timeline-title">Title 1</div>
            <div className="timeline-description">
              Lorem ipsum is a placeholder text commonly used to demonstrate the visual form of a document or a typeface without relying on meaningful content.
            </div>
          </div>
        </div>
        <div className="timeline-dot" style={{ left: '25%' }}>
          <div className="timeline-content">
            <div className="timeline-title">Title 2</div>
            <div className="timeline-description">
              Lorem ipsum is a placeholder text commonly used to demonstrate the visual form of a document or a typeface without relying on

The output was terrible. So, let's try providing it some help.

In [30]:
def get_text_chunks_langchain(text):
    text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=100)
    docs = [Document(page_content=x) for x in text_splitter.split_text(text)]
    return docs

extra_info = '''
Basic timeline example
Basic example of a responsive vertical timeline, with dates and event names. CSS-only (without JavaScript).

<!-- Section: Timeline -->
<section class="py-5">
  <ul class="timeline">
    <li class="timeline-item mb-5">
      <h5 class="fw-bold">Our company starts its operations</h5>
      <p class="text-muted mb-2 fw-bold">11 March 2020</p>
      <p class="text-muted">
        Lorem ipsum dolor sit amet consectetur adipisicing elit. Sit
        necessitatibus adipisci, ad alias, voluptate pariatur officia
        repellendus repellat inventore fugit perferendis totam dolor
        voluptas et corrupti distinctio maxime corporis optio?
      </p>
    </li>

    <li class="timeline-item mb-5">
      <h5 class="fw-bold">First customer</h5>
      <p class="text-muted mb-2 fw-bold">19 March 2020</p>
      <p class="text-muted">
        Quisque ornare dui nibh, sagittis egestas nisi luctus nec. Sed
        aliquet laoreet sapien, eget pulvinar lectus maximus vel.
        Phasellus suscipit porta mattis.
      </p>
    </li>

    <li class="timeline-item mb-5">
      <h5 class="fw-bold">Our team exceeds 10 people</h5>
      <p class="text-muted mb-2 fw-bold">24 June 2020</p>
      <p class="text-muted">
        Orci varius natoque penatibus et magnis dis parturient montes,
        nascetur ridiculus mus. Nulla ullamcorper arcu lacus, maximus
        facilisis erat pellentesque nec. Duis et dui maximus dui aliquam
        convallis. Quisque consectetur purus erat, et ullamcorper sapien
        tincidunt vitae.
      </p>
    </li>

    <li class="timeline-item mb-5">
      <h5 class="fw-bold">Earned the first million $!</h5>
      <p class="text-muted mb-2 fw-bold">15 October 2020</p>
      <p class="text-muted">
        Nulla ac tellus convallis, pulvinar nulla ac, fermentum diam. Sed
        et urna sit amet massa dapibus tristique non finibus ligula. Nam
        pharetra libero nibh, id feugiat tortor rhoncus vitae. Ut suscipit
        vulputate mattis.
      </p>
    </li>
  </ul>
</section>
<!-- Section: Timeline -->

.timeline {
  border-left: 1px solid hsl(0, 0%, 90%);
  position: relative;
  list-style: none;
}

.timeline .timeline-item {
  position: relative;
}

.timeline .timeline-item:after {
  position: absolute;
  display: block;
  top: 0;
}

.timeline .timeline-item:after {
  background-color: hsl(0, 0%, 90%);
  left: -38px;
  border-radius: 50%;
  height: 11px;
  width: 11px;
  content: "";
}

'''



docs = get_text_chunks_langchain(extra_info)

# # Load chunked documents into the FAISS index
db_extra = FAISS.from_documents(docs, HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

db.merge_from(db_extra)
retriever = db.as_retriever()

rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [31]:
question = '''
Generate a timeline component. It should be in a vertical fashion. The timeline should be a vertical line that stretches for 75% of the height.
Every timestamp should have a big dot on the timeline and should be evenly split across the timeline.
While placing the timestamps, start from the top and place them from top to bottom in a evenly spaced manner.
There should be 3 timestamps : "Jan 1, 2024", "May 1, 2024", "Jul 2, 2024". These dates should be placed horizontal to their correcsponding dots.
These are accent fonts (so have lighter font-color and smaller font-size).
Below that, there should be the title (this is the heading but have a small font anyway) that should say "Title 1", "Title 2" and "Title 3".
Below the title, there should be a small description that says " Lorem ipsum is a placeholder text commonly used to demonstrate the visual form of a document or a typeface without relying on meaningful content." for each of the timestamps.
In the timestamp corresponding to "Jan 1, 2024", have a CTA button below the description that says "Learn More"

Leave some horizontal margin from the timestamp dot to the timestamp content.
Make sure the title, description, button, etc (the content of the timestamp) is presented neatly with proper margins and paddings
Use a light grey background and proper font colors to make the UI look good.


Give the react component (JS not typescript) as output and follow the styles given in the context.
Define all the functions that you use in the component definition.
DO NOT USE ANY PROPS instead hardcode the values wherever necessary.

ALLOWED : import React from'react';
ALLOWED : import "styles.css";

NO OTHER IMPORTS ARE ALLOWED.
FOLLOW THE COLUMN SPANS USING BOOTSTRAP FLEX-BOX OR GRID.
GIVE PROPER PADDINGS AND MARGIN WHEREVER NECESSARY SO THAT THE UI LOOKS GOOD.
GIVE PROPER FONT-SIZE, FONT-COLOR AND OTHER FONT RELATED THINGS SO THAT THE UI LOOKS GOOD
GIVE PROPER COLORS, SHADOWS, ACCENTS, ETC SO THAT THE UI LOOKS GOOD.

'''

result = rag_chain.invoke(question)

print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```jsx
import React from'react';
import './styles.css';

const Timeline = () => {
  return (
    <div className="container">
      <div className="row">
        <div className="col-md-4">
          <div className="timeline">
            <div className="timeline-dot" style={{ backgroundColor: '#ff69b4' }}></div>
            <div className="timeline-content">
              <div className="timeline-title">Title 1</div>
              <div className="timeline-description">
                Lorem ipsum is a placeholder text commonly used to demonstrate the visual form of a document or a typeface without relying on meaningful content.
              </div>
              <div className="timeline-button">
                <button className="btn btn-primary">Learn More</button>
              </div>
            </div>
          </div>
        </div>
        <div className="col-md-4">
          <div className="timeline">
            <div className="timeline-dot" style={{ backgroundColor: '#008cba' }